In [1]:
import pandas as pd
import jieba

In [2]:
dz_df = pd.read_csv(u'GBT14396-2016.csv')
dz_df.head()

,SJBXX01,SJBXX02,PYM,SZM,WBM
0,A00.000,霍乱，由于O1群霍乱弧菌，霍乱生物型所致,HLYYO1QHLHJHLSWXSZ,1321O1513511333131,FTMGO1VFTXAFTTTGRG
1,A00.100,霍乱，由于O1群霍乱弧菌，埃尔托生物型所致,HLYYO1QHLHJAETSWXSZ,1321O15135113133131,FTMGO1VFTXAFQRTTGRG
2,A00.900,霍乱,HL,13,FT
3,A01.000,伤寒,SH,34,WP
4,A01.001+K77.0*,伤寒性肝炎,SHXGY,34234,WPNEO


In [3]:
dz_df['class'] = [i[:3] for i in dz_df['SJBXX01']]
dz_df.head()

,SJBXX01,SJBXX02,PYM,SZM,WBM,class
0,A00.000,霍乱，由于O1群霍乱弧菌，霍乱生物型所致,HLYYO1QHLHJHLSWXSZ,1321O1513511333131,FTMGO1VFTXAFTTTGRG,A00
1,A00.100,霍乱，由于O1群霍乱弧菌，埃尔托生物型所致,HLYYO1QHLHJAETSWXSZ,1321O15135113133131,FTMGO1VFTXAFQRTTGRG,A00
2,A00.900,霍乱,HL,13,FT,A00
3,A01.000,伤寒,SH,34,WP,A01
4,A01.001+K77.0*,伤寒性肝炎,SHXGY,34234,WPNEO,A01


In [4]:
dzclass_df = pd.DataFrame(dz_df[['class', 'SJBXX02']].values,columns=['class','name'])
dzclass_df.head()

,class,name
0,A00,霍乱，由于O1群霍乱弧菌，霍乱生物型所致
1,A00,霍乱，由于O1群霍乱弧菌，埃尔托生物型所致
2,A00,霍乱
3,A01,伤寒
4,A01,伤寒性肝炎


In [5]:
g = dzclass_df.groupby('class')
nitems_eachclass = g.size().values
# nitems_eachclass
# print(g.size())

In [6]:
classset = set(dzclass_df['class'].values)
print('疾病共 % d 类' % len(classset))

疾病共  2047 类


In [7]:
classlist = sorted(list(classset))
# print(classlist)

In [8]:
# strlist = []
# for index in classlist:
#     localstrlist = dzclass_df['name'][dzclass_df['class']==index].values
#     tempstr = ''
#     for s in localstrlist:
#         tempstr += s
#     strlist.append(tempstr)
# print(strlist)
# print(len(strlist))

In [9]:
print(len(classlist))
dzclass_df = dzclass_df.sort_values(by=['class'])  
# 因为之前对class 进行了排序,得到的nitems_eachclass是class排序后的对应结果, 所以这里要对dzclass_df重新排序
# dzclass_df

2047


In [10]:
'''较为快速的实现, 比上个版本快5倍'''
strlist = []
start_index = 0
for index in nitems_eachclass:
    localstrlist = dzclass_df['name'][start_index:start_index+index].values
    tempstr = ''
#     print(localstrlist)
    for s in localstrlist:
        tempstr += ' ' + s
    strlist.append(tempstr.strip())
    start_index += index
# print(strlist)
print(len(strlist))

2047


In [11]:
strlist[0]

'霍乱，由于O1群霍乱弧菌，霍乱生物型所致 霍乱，由于O1群霍乱弧菌，埃尔托生物型所致 霍乱'

In [12]:
from collections import Counter
seg_list = jieba.cut(strlist[1], cut_all=True)  # cut_all=True表示全模式。默认为精确模式
c = Counter()
for x in seg_list:
#     print(x)
    if len(x)>1 and x != '\r\n':  # 除去单字符以及换行(\r把光标移动到行首)
        c[x] += 1
print(c)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.716 seconds.
Prefix dict has been built succesfully.


Counter({'伤寒': 8, '副伤寒': 4, '败血': 1, '血症': 1, '脑膜': 1, '肝炎': 1, '伤寒杆菌': 1, '性肝炎': 1, '杆菌': 1, '败血症': 1, '膜炎': 1, '脑膜炎': 1})


In [15]:
jieba.load_userdict("disease_dict.txt")  # load my dict.txt

In [24]:
def wordscounter(strlist):
    dzlist = []
    for s in strlist:
        c = Counter()
        seg_list = jieba.cut(s, HMM=True)
        for x in seg_list:
            if len(x)>1 and x != '\r\n':  # 除去单字符以及换行(\r把光标移动到行首)
                c[x] += 1
        print(c.most_common(1))
        if c.most_common(1):
            dzlist.append(c.most_common(1)[0][0])
    return dzlist

In [16]:
dzlist = wordscounter(strlist)
dzset = set(dzlist)
print(len(dzlist), len(dzset))
print(dzset)

2046 1204
{'丧失', '恶性肿瘤其他治疗后的随诊检查', '链球菌作为分类于其他章疾病的原因', '检查', '别人', '呕吐', '前列腺', '支链氨基酸代谢紊乱', '虐待', '针刺', '闪电', '腺瘤', '不知', '二期梅毒', '摄入', '克山病', '麻风', '棘球', '急性', '乳房', '眼眶', '栓塞', '肾绞痛', '用爆炸物方式故意自害', '椅子', '恶性肿瘤个人史', '涎腺', '淋球菌', '土拉菌', '新生儿惊厥', '改变', '气胀', '利斯特', '河病', '先天性碘缺乏综合征', '恢复期', '陈旧性', '火焰', '毒性', '肾脏病', '梅毒', '支气管', '故意', '后遗症', '寄生虫', '轻度妊娠剧吐', '急性肾炎综合征', '鼠疫', '非性病性梅毒', '谵妄', '腕和手多处损伤', '阴道恶性肿瘤', '螺菌病', '耳漏', '皮肤棘层松解性疾患', '滑膜破裂', '免疫缺陷', '嗅觉', '运输车', '挤压', '大腿损伤', '蛋白尿', '类风湿性关节炎', '产程', '类药物', '用淹溺和沉没进行加害', '小头畸形', '尿道下裂', '肺水肿', '短暂性关节炎', '糖尿病前期', '慢性化脓性中耳炎胆脂瘤型', '溶血性贫血', '相关性', '结膜', '手术', '良性', '排尿里急后重', '生育问题', '霍奇金', '神经系统', '组织胞浆菌病', '原虫', '社会化的品行障碍', '操作', '语音障碍', '乳突', '正常', '心因性', '肠道寄生虫病', '复发性抑郁障碍', '导管', '产科', '调节', '蜂窝织炎', '46', '气管挤压伤', '缺铁性贫血', '少尿', '心肌病', '用钝器方式故意自害', '任何', '寻常疣', '食管', '贮积', '膀胱前壁恶性肿瘤', '多糖', '尔克', '脓肿', '葡萄胎', '健康', '肺气肿', '言语和语言发育障碍', '慢性放射性皮炎', '四肢', '给予', '乙醇的毒性效应', '葡萄球菌性烫伤样皮肤综合征', '风疹', '阿尔茨海默病性痴呆', '丘疹', '毛囊', '肾病综合征', '去纤维蛋白综合征

In [25]:
wordscounter(strlist[0:2])


[('霍乱', 4), ('O1', 2)]
[('伤寒', 2), ('伤寒性脑膜炎', 1)]


['霍乱', '伤寒']

In [80]:
dzname2 = pd.read_csv('faculty and disease name.csv')['disease']
ruleset={'尿', '便', '血', '病', '症', '肿', '疫', '风',
         '痫', '毒', '炎', '征' , '陷', '裂', '育', '瘤',
         '癌', '指', '热', '眼', '寒', '乱', '痘', '花'
        '癣', '疮', '疡', '痛', '退', '染', '障', '碍'}
removeset={'特指', '河病', '疾病', '血症', '缺陷',
           '侵染', '感染', '发育', '造血', '性贫血',
           '性疾病', '副指', '无毒', '为抗其他特指的单一的病毒性疾病采取必要的免疫',
           '医疗或生物材料被污染'
           }

def select(namelist):
    dzset = set()
    for name in namelist:
        if name.strip()[-1] in ruleset and name.strip() not in removeset:
            dzset.add(name)
    return dzset

In [81]:
dzset2 = set(dzname2.values)
dzneed2filter = dzset - dzset2 & dzset
# print(dzneed2filter)
print(len(dzneed2filter))
dzset3 = select(dzneed2filter)
print(len(dzset3))

1122
356


In [82]:
print(dzset3)

{'支链氨基酸代谢紊乱', '腺瘤', '克山病', '麻风', '肾绞痛', '先天性碘缺乏综合征', '肾脏病', '后遗症', '急性肾炎综合征', '鼠疫', '非性病性梅毒', '阴道恶性肿瘤', '螺菌病', '滑膜破裂', '免疫缺陷', '蛋白尿', '尿道下裂', '短暂性关节炎', '溶血性贫血', '组织胞浆菌病', '社会化的品行障碍', '语音障碍', '肠道寄生虫病', '复发性抑郁障碍', '蜂窝织炎', '少尿', '心肌病', '膀胱前壁恶性肿瘤', '言语和语言发育障碍', '慢性放射性皮炎', '葡萄球菌性烫伤样皮肤综合征', '去纤维蛋白综合征', '接触性皮炎', '皮肤良性肿瘤', '肌腱端病', '并指', '舌根背面恶性肿瘤', '先天性鱼鳞病', '虱病', '性脓皮症', '分裂情感性障碍', '糖尿', '气管恶性肿瘤', '脑血管病后遗症', '嗜酸细胞性肺炎', '疱疹病毒', '登革热', '感应性妄想性障碍', '虹膜睫状体炎', '肉芽肿', '双相情感障碍', '角膜溃疡', '蝶窦恶性肿瘤', '大肠杆菌感染', '型糖尿病', '脑炎', '产褥期脓毒病', '婴儿猝死综合征', '真菌病', '心理发育障碍', '帕金森病', '内出血', '视觉障碍', '新生儿脐炎', '骨髓增生异常综合征', '子宫内低氧症', '囊肿', '肝病', '蛋白血症', '极严重的酒精中毒', '黄热病', '膝关节病', '多关节炎', '胎儿失血', '慢性会厌炎', '前庭大腺恶性肿瘤', '城市狂犬病', '混合型和其他人格障碍', '品行和情绪混合性障碍', '慢性牙周炎', '佝偻病', '甲状舌管良性肿瘤', '雅司病', '肿瘤', '副球孢子菌病', '特定性运动功能发育障碍', '急性淋巴结炎', '卡波西肉瘤', '综合征', '静脉炎', '依法处置的后遗症', '动脉破裂', '新生儿出血', '性身份障碍', '腮腺恶性肿瘤', '叶酸盐缺乏性贫血', '全身炎症反应综合征', '丹毒', '类天疱疮', '病毒性肝炎', '结缔组织和软组织恶性肿瘤', '子宫恶性肿瘤', '结节病', '缺乏症', '睑腺炎', '病毒性脑膜炎', '螺旋体感染', '胎儿水肿', '虐待综合

In [83]:
finaldzset = (dzname for dzname in dzset2|dzset3)
with open('disease name v0.1.txt', 'w') as f:
    for i in finaldzset:
        f.write(i+'\n')
        
print(len(dzset2|dzset3))


895
